<a href="https://colab.research.google.com/github/ankitstar01/ml-learning/blob/master/smart_compose1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#importing os environment
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/Kaggle"
%cd Kaggle/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


TIMEOUT: ignored

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import email
import re
import shutil
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, LSTM, Flatten, TimeDistributed, Dropout, LSTMCell, RNN, Bidirectional, Concatenate, Layer
from keras import Model

In [ ]:
shutil.copyfile("/content/drive/My Drive/Colab Notebooks/globe.zip","/content/globe.zip")

!mkdir Kaggle
shutil.copyfile("/content/drive/My Drive/Colab Notebooks/kaggle.json","/content/Kaggle/kaggle.json")
#downloading the kaggle dataset
!kaggle datasets download -d wcukierski/enron-email-dataset

#unzipping the zip files
!unzip \*.zip


In [ ]:
# shutil.move("/content/gdrive/My Drive/Kaggle/emails.csv","/content/emails.csv")
#reading the emails
if not 'emails_df' in locals():
    emails_df = pd.read_csv('/content/emails.csv')
#printing the matrix size and the top 5 rows of it
print(emails_df.shape)
emails_df.head()

In [ ]:
# Convert to message objects from the message strings
messages = list(map(email.message_from_string, emails_df['message']))

In [ ]:
def get_text_from_email(msg):
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    text = ''.join(parts)
    return text

emails = pd.DataFrame()

# Parse content from emails
emails['content'] = list(map(get_text_from_email, messages))
print(messages[:10])
print(emails['content'][:10])

In [ ]:
import gc

# Remove variables from memory
del messages
del emails_df
gc.collect()

In [ ]:
def normalize_text(text):
    text = text.lower()
  
  # creating a space between a word and the punctuation following it to separate words
  # and compact repetition of punctuation
  # eg: "he is a boy.." => "he is a boy ."
    text = re.sub(r'([.,!?]+)', r" \1 ", text)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,']+", " ", text)

  # Compact spaces
    text = re.sub(r'[" "]+', " ", text)

  # Remove forwarded messages
    text = text.split('forwarded by')[0]

    text = text.strip()

    return text

emails['content'] = list(map(normalize_text, emails['content']))

In [ ]:
# Drop samples with empty content text after normalization
emails['content'].replace('', np.nan, inplace=True)
emails.dropna(subset=['content'], inplace=True)

In [ ]:
#some sample emails
pd.set_option('max_colwidth', -1)
emails

In [ ]:
# Skip long sentences, which increase maximum length a lot when padding
# and make the number of parameters to train explode
SENTENCE_MAX_WORDS = 20

def generate_dataset (emails):
    contents = emails['content']
    output = []
    vocabulary_sentences = []
    i=0
    for content in contents:
        # Skip emails longer than one sentence
        i=i+1
        if (len(content) > SENTENCE_MAX_WORDS * 5):
            continue

        sentences = content.split('. ')
        for sentence in sentences:
            # Remove user names from start or end of sentence. This is just an heuristic
            # but it's more efficient than compiling the list of names and removing all of them
            sentence = re.sub("(^\w+\s,\s)|(\s,\s\w+$)", "", sentence)
            words = sentence.split(' ')
            
            if ((len(words) > SENTENCE_MAX_WORDS) or (len(words) < 2)):
                continue

            vocabulary_sentences.append('<start> ' + sentence + ' <end>')
            for i in range(1, len(words) - 1):
                input_data = '<start> ' + ' '.join(words[:i+1]) + ' <end>'
                output_data = '<start> ' + ' '.join(words[i+1:]) + ' <end>'
                data = (input_data, output_data)
                output.append(data)
    print("Dataset prepared with prefix and suffixes for teacher forcing technique")
    dummy_df = pd.DataFrame(output, columns=['input','output'])
    return output, dummy_df


class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))
        self.vocab = sorted(self.vocab)
        self.word2idx["<pad>"] = 0
        self.idx2word[0] = "<pad>"
        for i,word in enumerate(self.vocab):
            self.word2idx[word] = i + 1
            self.idx2word[i+1] = word

def max_length(t):
    return max(len(i) for i in t)

def load_dataset():
    pairs,df = generate_dataset(emails)
    out_lang = LanguageIndex(sp for en, sp in pairs)
    in_lang = LanguageIndex(en for en, sp in pairs)
    input_data = [[in_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    output_data = [[out_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]

    max_length_in, max_length_out = max_length(input_data), max_length(output_data)
    input_data = tf.keras.preprocessing.sequence.pad_sequences(input_data, maxlen=max_length_in, padding="post")
    output_data = tf.keras.preprocessing.sequence.pad_sequences(output_data, maxlen=max_length_out, padding="post")
    return input_data, output_data, in_lang, out_lang, max_length_in, max_length_out, df
input_data, teacher_data, input_lang, target_lang, len_input, len_target, df = load_dataset()

